In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
# from transformers import LlamaTokenizer

In [3]:
tokenizer_70b = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-hf")
df = pd.read_csv("../02-data/results_GPT.csv")
shapes = ["triangle", "circle", "hexagon", "square"]
colors = ["red", "blue", "green", "orange"]
texture = ["stripes", "spades", "dots", "stars"]

context = df.loc[0, 'context_production']
context

'Your task is to play a conversation game. There are three objects that you and your friend can see. You have to choose a single word to identify one of the three objects for your friend.\n\nThe three objects are:\n\na green triangle with dots\na green circle with dots\na red triangle with dots\n\nYour task is to make your friend pick out the following target object:\n\nthe red triangle with dots\n\nWhich of the following words would you choose:\n\ngreen\ntriangle\nred\ncircle\n\nYour answer:\n\nI would choose the word '

In [14]:
len_context = len(tokenizer_70b(context.strip())['input_ids'])
print("Len context ", len_context)
tokens_shapes = [
    len(tokenizer_70b(context + '"' + w + '"')['input_ids']) - len_context 
    for w in shapes
]
tokens_shapes

Len context  126


[3, 3, 4, 3]

In [11]:
tokens_colors = [
    len(tokenizer_70b(context + '"' + w + '"')['input_ids']) - len_context 
    for w in colors
]
tokens_colors

[3, 3, 3, 4]

In [12]:
tokens_texture = [
    len(tokenizer_70b(context + '"' + w + '"')['input_ids']) - len_context 
    for w in texture
]
tokens_texture

[5, 4, 3, 4]

In [17]:
tokens_shapes = {s: t for s, t in zip(shapes, tokens_shapes)}
tokens_colors = {s: t for s, t in zip(colors, tokens_colors)}
tokens_texture = {s: t for s, t in zip(texture, tokens_texture)}

In [32]:
all_tokens = {**tokens_shapes, **tokens_colors, **tokens_texture}
all_tokens

{'triangle': 3,
 'circle': 3,
 'hexagon': 4,
 'square': 3,
 'red': 3,
 'blue': 3,
 'green': 3,
 'orange': 4,
 'stripes': 5,
 'spades': 4,
 'dots': 3,
 'stars': 4}

In [21]:
# compare free generation and scoring results with the lens of the number of tokens
df70 = pd.read_csv("results_wSamples_greedyDecode_data_Llama-2-70b-hf_20240310_2312.csv") 
df13 = pd.read_csv("results_wQuots_wSamples_greedyDecode_data_Llama-2-13b-hf_20240311_0918.csv")
# TODO: count argmax selected options

In [19]:
df70.columns

Index(['trial', 'utterances', 'trigger_feature', 'nuisance_feature',
       'production_target', 'production_competitor', 'production_distractor1',
       'production_distractor2', 'production_index_target',
       'production_index_competitor', 'production_index_distractor1',
       'production_index_distractor2', 'trigger_object', 'trigger_word',
       'interpretation_target', 'interpretation_competitor',
       'interpretation_distractor', 'interpretation_index_target',
       'interpretation_index_competitor', 'interpretation_index_distractor',
       'context_production', 'context_interpretation', 'alpha_production',
       'scores_production_target', 'scores_production_competitor',
       'scores_production_distractor1', 'scores_production_distractor2',
       'scores_production_target_npnlg', 'scores_production_competitor_npnlg',
       'scores_production_distractor1_npnlg',
       'scores_production_distractor2_npnlg', 'prob_production_target',
       'prob_production_competit

In [27]:
def pull_sampled_option(s):
    continuation = s['production_decoded'].replace(s["context_production"], "").replace("<s> ", "")
    selected_word = continuation.split(" ")[0]
    return selected_word

production_cols = ["scores_production_target", "scores_production_competitor", "scores_production_distractor1", "scores_production_distractor2"]
production_options = ["target", "competitor", "distractor1", "distractor2"]
def pull_argmax_option(s):
    scores = [s[c] for c in production_cols]
    option = production_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'production_' + option
    gt_option = s[option_name]
    return gt_option

def pull_argmax(s):
    scores = [s[c] for c in production_cols]
    option = production_cols[np.argmax(scores)].split("_")[-1]
    return option


df70['sampled_option'] = df70.apply(lambda r: pull_sampled_option(r), axis=1)
df13['sampled_option'] = df13.apply(lambda r: pull_sampled_option(r), axis=1)

df70['argmax_option'] = df70.apply(lambda r: pull_argmax_option(r), axis=1)
df13['argmax_option'] = df13.apply(lambda r: pull_argmax_option(r), axis=1)

df70["argmax"] = df70.apply(lambda r: pull_argmax(r), axis=1)
df13["argmax"] = df13.apply(lambda r: pull_argmax(r), axis=1)

In [28]:
df70.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,scores_interpretation_target_npnlg,scores_interpretation_competitor_npnlg,scores_interpretation_distractor_npnlg,prob_interpretation_target_npnlg,prob_interpretation_competitor_npnlg,prob_interpretation_distractor_npnlg,interpretation_decoded,sampled_option,argmax_option,argmax
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,1.272613,1.525008,1.736368,0.257922,0.331973,0.410105,<s> Your task is to play a conversation game. ...,"""triangle"".\n\nYour",circle,distractor2
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,1.221797,1.292234,1.757923,0.264386,0.283680,0.451934,<s> Your task is to play a conversation game. ...,"""stripes""",hexagon,competitor
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,1.237588,1.338990,1.548762,0.288039,0.318779,0.393181,<s> Your task is to play a conversation game. ...,"""orange"".\n\nYour",orange,target
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,1.580697,1.715218,1.948243,0.278693,0.318822,0.402485,<s> Your task is to play a conversation game. ...,"""orange""",orange,competitor
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,1.563322,1.165484,1.338524,0.404785,0.271923,0.323292,<s> Your task is to play a conversation game. ...,"""hexagon""",hexagon,target


In [29]:
df13.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,scores_interpretation_target_npnlg,scores_interpretation_competitor_npnlg,scores_interpretation_distractor_npnlg,prob_interpretation_target_npnlg,prob_interpretation_competitor_npnlg,prob_interpretation_distractor_npnlg,interpretation_decoded,sampled_option,argmax_option,argmax
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,1.093133,1.140121,1.507981,0.280721,0.294227,0.425052,<s> Your task is to play a conversation game. ...,"""triangle""",triangle,competitor
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,0.885817,1.313568,1.372629,0.240331,0.368621,0.391048,<s> Your task is to play a conversation game. ...,"""stars""\n\nYour",stars,target
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,1.174740,1.022200,1.390098,0.322702,0.277048,0.400250,<s> Your task is to play a conversation game. ...,"""orange""",orange,target
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,1.314342,1.398015,1.357680,0.319329,0.347198,0.333472,<s> Your task is to play a conversation game. ...,"""orange""",orange,competitor
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,1.384628,1.227970,1.253190,0.366055,0.312976,0.320969,<s> Your task is to play a conversation game. ...,"""stars""",stars,competitor


In [30]:
from collections import Counter

Counter(df70['argmax'].tolist())

Counter({'distractor2': 10, 'competitor': 39, 'target': 37, 'distractor1': 14})

In [31]:
Counter(df13['argmax'].tolist())

Counter({'competitor': 34, 'target': 65, 'distractor1': 1})

In [50]:
def check_match(s, o1='argmax_option', o2='sampled_option'):
    return s[o1] in s[o2]

def pull_tokens(s, all_tokens=all_tokens):
    for k,v in all_tokens.items():
        if k in s:
            return v
        
df70["match"] = df70.apply(lambda r: check_match(r), axis=1)
df70["sampled_option_tokens"] = df70['sampled_option'].apply(lambda r: pull_tokens(r))
df70["argmax_option_tokens"] = df70['argmax_option'].apply(lambda r: pull_tokens(r))

df13["match"] = df13.apply(lambda r: check_match(r), axis=1)
df13["sampled_option_tokens"] = df13['sampled_option'].apply(lambda r: pull_tokens(r))
df13["argmax_option_tokens"] = df13['argmax_option'].apply(lambda r: pull_tokens(r))

In [34]:
df70.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,prob_interpretation_target_npnlg,prob_interpretation_competitor_npnlg,prob_interpretation_distractor_npnlg,interpretation_decoded,sampled_option,argmax_option,argmax,match,sampled_option_tokens,argmax_option_tokens
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,0.257922,0.331973,0.410105,<s> Your task is to play a conversation game. ...,"""triangle"".\n\nYour",circle,distractor2,False,3,3
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,0.264386,0.283680,0.451934,<s> Your task is to play a conversation game. ...,"""stripes""",hexagon,competitor,False,5,4
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,0.288039,0.318779,0.393181,<s> Your task is to play a conversation game. ...,"""orange"".\n\nYour",orange,target,True,4,4
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,0.278693,0.318822,0.402485,<s> Your task is to play a conversation game. ...,"""orange""",orange,competitor,True,4,4
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,0.404785,0.271923,0.323292,<s> Your task is to play a conversation game. ...,"""hexagon""",hexagon,target,True,4,4


In [35]:
df13.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,prob_interpretation_target_npnlg,prob_interpretation_competitor_npnlg,prob_interpretation_distractor_npnlg,interpretation_decoded,sampled_option,argmax_option,argmax,match,sampled_option_tokens,argmax_option_tokens
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,0.280721,0.294227,0.425052,<s> Your task is to play a conversation game. ...,"""triangle""",triangle,competitor,True,3,3
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,0.240331,0.368621,0.391048,<s> Your task is to play a conversation game. ...,"""stars""\n\nYour",stars,target,True,4,4
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,0.322702,0.277048,0.400250,<s> Your task is to play a conversation game. ...,"""orange""",orange,target,True,4,4
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,0.319329,0.347198,0.333472,<s> Your task is to play a conversation game. ...,"""orange""",orange,competitor,True,4,4
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,0.366055,0.312976,0.320969,<s> Your task is to play a conversation game. ...,"""stars""",stars,competitor,True,4,4


In [39]:
print(df70['match'].mean())
df70['tokens_match'] = df70['sampled_option_tokens'] == df70['argmax_option_tokens']
print(df70['tokens_match'].mean())
df70['match_corr'] = df70['match'] == df70['tokens_match']
print(df70['match_corr'].mean())

0.53
0.68
0.85


In [40]:
print(df13['match'].mean())
df13['tokens_match'] = df13['sampled_option_tokens'] == df13['argmax_option_tokens']
print(df13['tokens_match'].mean())
df13['match_corr'] = df13['match'] == df13['tokens_match']
print(df13['match_corr'].mean())

0.95
0.97
0.98


In [49]:
df70["target_tokens"] = df70['production_target'].apply(lambda r: pull_tokens(r))
df70["competitor_tokens"] = df70['production_competitor'].apply(lambda r: pull_tokens(r))
df70["distractor1_tokens"] = df70['production_distractor1'].apply(lambda r: pull_tokens(r))
df70["distractor2_tokens"] = df70['production_distractor2'].apply(lambda r: pull_tokens(r))

def avg_score(r, option='target'):
    opt = 'scores_production_' + option
    return r[opt] / r[option + '_tokens']

df70['avg_score_target'] = df70.apply(lambda r: avg_score(r, option='target'), axis=1)
df70['avg_score_competitor'] = df70.apply(lambda r: avg_score(r, option='competitor'), axis=1)
df70['avg_score_distractor1'] = df70.apply(lambda r: avg_score(r, option='distractor1'), axis=1)
df70['avg_score_distractor2'] = df70.apply(lambda r: avg_score(r, option='distractor2'), axis=1)

avg_cols = ["avg_score_target", "avg_score_competitor", "avg_score_distractor1", "avg_score_distractor2"]
def pull_argmax_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    return option

def pull_argmax_option_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'production_' + option
    gt_option = s[option_name]
    return gt_option

df70["argmax_avg"] = df70.apply(lambda r: pull_argmax_avg(r), axis=1)
df70["argmax_option_avg"] = df70.apply(lambda r: pull_argmax_option_avg(r), axis=1)

In [46]:
df70.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,match_corr,target_tokens,competitor_tokens,distractor1_tokens,distractor2_tokens,avg_score_target,avg_score_competitor,avg_score_distractor1,avg_score_distractor2,argmax_avg
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,False,3,3,3,3,-0.874843,-0.893072,-0.895677,-0.869635,distractor2
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,True,4,4,5,3,-0.727306,-0.642395,-0.826514,-0.894221,competitor
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,True,4,5,3,4,-0.578006,-0.645087,-1.156091,-0.883604,target
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,True,3,4,3,3,-1.134577,-0.561870,-1.035619,-1.137181,competitor
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,True,4,4,3,3,-0.617829,-0.634668,-1.351432,-1.110313,target


In [48]:
Counter(df70['argmax_avg'].tolist())

Counter({'distractor2': 10, 'competitor': 32, 'target': 38, 'distractor1': 20})

In [51]:
df70["gen_avg_match"] = df70.apply(lambda s: check_match(s, "argmax_option_avg", "sampled_option"), axis=1)
df70["gen_avg_match"].mean()

0.45

In [52]:
# do the same exercise with 13b results
df13["target_tokens"] = df13['production_target'].apply(lambda r: pull_tokens(r))
df13["competitor_tokens"] = df13['production_competitor'].apply(lambda r: pull_tokens(r))
df13["distractor1_tokens"] = df13['production_distractor1'].apply(lambda r: pull_tokens(r))
df13["distractor2_tokens"] = df13['production_distractor2'].apply(lambda r: pull_tokens(r))

df13['avg_score_target'] = df13.apply(lambda r: avg_score(r, option='target'), axis=1)
df13['avg_score_competitor'] = df13.apply(lambda r: avg_score(r, option='competitor'), axis=1)
df13['avg_score_distractor1'] = df13.apply(lambda r: avg_score(r, option='distractor1'), axis=1)
df13['avg_score_distractor2'] = df13.apply(lambda r: avg_score(r, option='distractor2'), axis=1)

df13["argmax_avg"] = df13.apply(lambda r: pull_argmax_avg(r), axis=1)
df13["argmax_option_avg"] = df13.apply(lambda r: pull_argmax_option_avg(r), axis=1)
print("Updated counts with avg scores: ", Counter(df13['argmax_avg'].tolist()))
df13["gen_avg_match"] = df13.apply(lambda s: check_match(s, "argmax_option_avg", "sampled_option"), axis=1)
df13["gen_avg_match"].mean()

Updated counts with avg scores:  Counter({'target': 65, 'competitor': 32, 'distractor1': 3})


0.77

In [54]:
def map_sample_onto_option(r):
    opt_val = {}
    for o in production_options:
        k = r["production_" + o]
        opt_val[k] = o
    sampled_option_clean = r["sampled_option"].replace('"', "").replace(".", "").split("\n\n")[0]
    sampled_value = opt_val[sampled_option_clean]
    return sampled_value

df70["sampled_option_mapped"] = df70.apply(lambda r: map_sample_onto_option(r), axis=1)

In [55]:
Counter(df70["sampled_option_mapped"].tolist())

Counter({'competitor': 38, 'distractor1': 14, 'target': 46, 'distractor2': 2})

In [56]:
df13["sampled_option_mapped"] = df13.apply(lambda r: map_sample_onto_option(r), axis=1)
Counter(df13["sampled_option_mapped"].tolist())

Counter({'competitor': 33, 'target': 66, 'distractor1': 1})

In [57]:
df70.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,distractor2_tokens,avg_score_target,avg_score_competitor,avg_score_distractor1,avg_score_distractor2,argmax_avg,match_prod_and_avg,argmax_option_avg,gen_avg_match,sampled_option_mapped
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,3,-0.874843,-0.893072,-0.895677,-0.869635,distractor2,True,circle,False,competitor
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,3,-0.727306,-0.642395,-0.826514,-0.894221,competitor,True,hexagon,False,distractor1
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,4,-0.578006,-0.645087,-1.156091,-0.883604,target,True,orange,True,target
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,3,-1.134577,-0.561870,-1.035619,-1.137181,competitor,True,orange,True,competitor
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,3,-0.617829,-0.634668,-1.351432,-1.110313,target,True,hexagon,True,target


In [59]:
# check if the mean and generated predictions are the same
(df70["sampled_option_mapped"] == df70["argmax_avg"]).mean()

0.45

In [61]:
any(df70["sampled_option_mapped"].isna())

False

In [66]:
# explore the effect of sum vs mean scoring under the chat results 
# NOTE: the tokenization under the INST prompt is different
context_chat = "[INST]"+context+ "[/INST]"
tokenizer_70b_chat = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")
len_context_chat = len(tokenizer_70b_chat(context_chat.strip())['input_ids'])
df70_chat = pd.read_csv("../02-data/results_Llama-2-70b-chat-hf.csv")

In [64]:
tokens_shapes_chat = [
    len(tokenizer_70b_chat(context_chat + '"' + w + '"')['input_ids']) - len_context_chat 
    for w in shapes
]
tokens_colors_chat = [
    len(tokenizer_70b_chat(context_chat + '"' + w + '"')['input_ids']) - len_context_chat 
    for w in colors
]
tokens_texture_chat = [
    len(tokenizer_70b_chat(context_chat + '"' + w + '"')['input_ids']) - len_context_chat 
    for w in texture
]
tokens_shapes_c = {s: t for s, t in zip(shapes, tokens_shapes_chat)}
tokens_colors_c = {s: t for s, t in zip(colors, tokens_colors_chat)}
tokens_texture_c = {s: t for s, t in zip(texture, tokens_texture_chat)}
tokens_all_chat = {**tokens_shapes_c, **tokens_colors_c, **tokens_texture_c}

In [68]:
df70_chat['argmax_option'] = df70_chat.apply(lambda r: pull_argmax_option(r), axis=1)
df70_chat["argmax"] = df70_chat.apply(lambda r: pull_argmax(r), axis=1)
df70_chat["argmax_option_tokens"] = df70['argmax_option'].apply(lambda r: pull_tokens(r))

In [74]:
Counter(df70_chat['argmax'].tolist())

Counter({'competitor': 54, 'target': 41, 'distractor2': 2, 'distractor1': 3})

In [72]:
df70_chat["target_tokens"] = df70_chat['production_target'].apply(lambda r: pull_tokens(r))
df70_chat["competitor_tokens"] = df70_chat['production_competitor'].apply(lambda r: pull_tokens(r))
df70_chat["distractor1_tokens"] = df70_chat['production_distractor1'].apply(lambda r: pull_tokens(r))
df70_chat["distractor2_tokens"] = df70_chat['production_distractor2'].apply(lambda r: pull_tokens(r))

def avg_score(r, option='target'):
    opt = 'scores_production_' + option
    return r[opt] / r[option + '_tokens']

df70_chat['avg_score_target'] = df70_chat.apply(lambda r: avg_score(r, option='target'), axis=1)
df70_chat['avg_score_competitor'] = df70_chat.apply(lambda r: avg_score(r, option='competitor'), axis=1)
df70_chat['avg_score_distractor1'] = df70_chat.apply(lambda r: avg_score(r, option='distractor1'), axis=1)
df70_chat['avg_score_distractor2'] = df70_chat.apply(lambda r: avg_score(r, option='distractor2'), axis=1)

avg_cols = ["avg_score_target", "avg_score_competitor", "avg_score_distractor1", "avg_score_distractor2"]
def pull_argmax_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    return option

def pull_argmax_option_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'production_' + option
    gt_option = s[option_name]
    return gt_option

df70_chat["argmax_avg"] = df70_chat.apply(lambda r: pull_argmax_avg(r), axis=1)
df70_chat["argmax_option_avg"] = df70_chat.apply(lambda r: pull_argmax_option_avg(r), axis=1)

In [73]:
Counter(df70_chat['argmax_avg'].tolist())

Counter({'competitor': 50, 'target': 44, 'distractor2': 3, 'distractor1': 3})

In [76]:
(df70_chat['argmax_avg'] == df70_chat['argmax']).mean()

0.9

In [77]:
(df13['argmax_avg'] == df13['argmax']).mean()

0.77

In [79]:
# do the avg aggregation for 7b base results just to check approach against re-run results
df7 = pd.read_csv("../02-data/results_Llama-2-7b-hf.csv")


In [81]:
df7['argmax_option'] = df7.apply(lambda r: pull_argmax_option(r), axis=1)
df7["argmax"] = df7.apply(lambda r: pull_argmax(r), axis=1)
df7["argmax_option_tokens"] = df7['argmax_option'].apply(lambda r: pull_tokens(r))

print("Sum aggregation: ", Counter(df7['argmax'].tolist()))

df7["target_tokens"] = df7['production_target'].apply(lambda r: pull_tokens(r))
df7["competitor_tokens"] = df7['production_competitor'].apply(lambda r: pull_tokens(r))
df7["distractor1_tokens"] = df7['production_distractor1'].apply(lambda r: pull_tokens(r))
df7["distractor2_tokens"] = df7['production_distractor2'].apply(lambda r: pull_tokens(r))


df7['avg_score_target'] = df7.apply(lambda r: avg_score(r, option='target'), axis=1)
df7['avg_score_competitor'] = df7.apply(lambda r: avg_score(r, option='competitor'), axis=1)
df7['avg_score_distractor1'] = df7.apply(lambda r: avg_score(r, option='distractor1'), axis=1)
df7['avg_score_distractor2'] = df7.apply(lambda r: avg_score(r, option='distractor2'), axis=1)

avg_cols = ["avg_score_target", "avg_score_competitor", "avg_score_distractor1", "avg_score_distractor2"]
def pull_argmax_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    return option

def pull_argmax_option_avg(s):
    scores = [s[c] for c in avg_cols]
    option = avg_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'production_' + option
    gt_option = s[option_name]
    return gt_option

df7["argmax_avg"] = df7.apply(lambda r: pull_argmax_avg(r), axis=1)
df7["argmax_option_avg"] = df7.apply(lambda r: pull_argmax_option_avg(r), axis=1)

print("Avg aggregation: ", Counter(df7['argmax_avg'].tolist()))

Sum aggregation:  Counter({'target': 62, 'competitor': 29, 'distractor1': 5, 'distractor2': 4})
Avg aggregation:  Counter({'target': 51, 'competitor': 29, 'distractor1': 10, 'distractor2': 10})


In [82]:
df7.head()

,trial,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,production_index_competitor,...,target_tokens,competitor_tokens,distractor1_tokens,distractor2_tokens,avg_score_target,avg_score_competitor,avg_score_distractor1,avg_score_distractor2,argmax_avg,argmax_option_avg
0,0,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,1,...,3,3,3,3,-1.124950,-1.236643,-1.424346,-1.264298,target,red
1,1,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,3,...,4,4,5,3,-0.674005,-1.110701,-0.697091,-1.672764,target,stars
2,2,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,2,...,4,5,3,4,-0.812075,-0.822653,-1.641342,-1.375972,target,orange
3,3,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,2,...,3,4,3,3,-1.538329,-0.892267,-1.186984,-1.569372,competitor,orange
4,4,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,3,...,4,4,3,3,-1.061446,-0.615450,-2.115889,-1.330520,competitor,stars


In [83]:
# analyse 7b base BoolQ results
boolq7b = pd.read_csv("boolq_ownMeanScores_greedySamples_wQuots_use_own_scoring_Llama-2-7b-hf_20240314_1036.csv")

In [87]:
boolq_cols = ["scores_target", "scores_distractor"]
boolq_options = ["correct", "incorrect"]

def pull_boolq_argmax_option(s):
    scores = [s[c] for c in boolq_cols]
    option = boolq_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'correct' if option == "target" else "incorrect"
    gt_option = s[option_name]
    return gt_option

def pull_boolq_argmax(s):
    scores = [s[c] for c in boolq_cols]
    option = boolq_cols[np.argmax(scores)].split("_")[-1]
    option_name = 'correct' if option == "target" else "incorrect"
    return option_name

boolq7b['argmax_option'] = boolq7b.apply(lambda r: pull_boolq_argmax_option(r), axis=1)
boolq7b['argmax'] = boolq7b.apply(lambda r: pull_boolq_argmax(r), axis=1)

boolq7b.head()

,trial,context,question,correct,incorrect,scores_target,scores_target_npnlg,generation_correct,generation_score_correct,scores_distractor,scores_distractor_npnlg,generation_distractor,generation_score_distractor,production_decoded,argmax_option,argmax
0,0,English Football League play-offs -- Before th...,do away goals count in the league 2 playoffs,No,Yes,-1.075884,1.075884,NONE,NaN,-1.141196,1.141196,NONE,NaN,<s> Context: English Football League play-offs...,No,correct
1,1,1986 NBA Playoffs -- Second-year player Michae...,did the bulls get swept by the celtics,Yes,No,-0.940614,0.940614,NONE,NaN,-1.030350,1.030350,NONE,NaN,<s> Context: 1986 NBA Playoffs -- Second-year ...,Yes,correct
2,2,Water table -- The groundwater may be from pre...,is the depth of the water table always the same,No,Yes,-0.914282,0.914283,NONE,NaN,-1.040346,1.040346,NONE,NaN,<s> Context: Water table -- The groundwater ma...,No,correct
3,3,FIFA World Cup qualification -- The hosts of t...,does the host country for the world cup get an...,Yes,No,-0.918134,0.918134,NONE,NaN,-1.027473,1.027473,NONE,NaN,<s> Context: FIFA World Cup qualification -- T...,Yes,correct
4,4,"Bad Lip Reading -- In December of 2015, Bad Li...",is seagulls stop it now a real song,Yes,No,-1.037458,1.037458,NONE,NaN,-1.217039,1.217039,NONE,NaN,<s> Context: Bad Lip Reading -- In December of...,Yes,correct


In [89]:
Counter(boolq7b['argmax'].tolist())
100/146

0.684931506849315

In [98]:
from llama_logprobs import soft_max

def pull_interpretation_tokens(row, tokenizer, option="target"):
    """
    Helper for retrieving number the number of tokens corresponding to different choice options in interpretation task.
    """
    context = row['context_interpretation']
    len_context = len(tokenizer(context.strip())['input_ids'])
    print("Len context ", len_context)
    tokens_shape = (
        len(tokenizer(context + '"' + row["interpretation_" + option] + '"')['input_ids']) - len_context 
    )
    print("tokens shape ", tokens_shape)
    return tokens_shape

def avg_score(r, option='production_target'):
    opt = 'scores_' + option
    return r[opt] / r[option + '_tokens']

def compute_row_probabilities(row):
    # production
    row["prob_production_target"], row["prob_production_competitor"], row["prob_production_distractor1"], row["prob_production_distractor2"] = soft_max(
        np.array([row["scores_production_target"], row["scores_production_competitor"], row["scores_production_distractor1"], row["scores_production_distractor2"]]),
        row["alpha_production"]
    ) 
    # interpretation
    row["prob_interpretation_target"], row["prob_interpretation_competitor"], row["prob_interpretation_distractor"] = soft_max(
        np.array([row["scores_interpretation_target"], row["scores_interpretation_competitor"], row["scores_interpretation_distractor"]]),
        row["alpha_interpretation"]
    )
    return row

def get_average_scores(filename, tokenizer):
    """
    Helper for creating results files with average scores instead of summed scores
    """
    raw_df = pd.read_csv(filename)
    df = raw_df.copy()
    # use numbers of tokens which correspond to tokenization with [INST]
    if "chat" in filename:
        tokens_counts = tokens_all_chat
    else:
        tokens_counts = all_tokens

    # pull token counts for each option
    df["production_target_tokens"] = df['production_target'].apply(lambda r: pull_tokens(r, tokens_counts))
    df["production_competitor_tokens"] = df['production_competitor'].apply(lambda r: pull_tokens(r, tokens_counts))
    df["production_distractor1_tokens"] = df['production_distractor1'].apply(lambda r: pull_tokens(r, tokens_counts))
    df["production_distractor2_tokens"] = df['production_distractor2'].apply(lambda r: pull_tokens(r, tokens_counts))
    df["interpretation_target_tokens"] = df.apply(lambda r: pull_interpretation_tokens(r, tokenizer, "target"), axis=1)
    df["interpretation_competitor_tokens"] = df.apply(lambda r: pull_interpretation_tokens(r, tokenizer, "competitor"), axis=1)
    df["interpretation_distractor_tokens"] = df.apply(lambda r: pull_interpretation_tokens(r, tokenizer, "distractor"), axis=1)

    # use the numbers of tokens in respective option to compute the average scores
    df['scores_production_target'] = df.apply(lambda r: avg_score(r, option='production_target'), axis=1)
    df['scores_production_competitor'] = df.apply(lambda r: avg_score(r, option='production_competitor'), axis=1)
    df['scores_production_distractor1'] = df.apply(lambda r: avg_score(r, option='production_distractor1'), axis=1)
    df['scores_production_distractor2'] = df.apply(lambda r: avg_score(r, option='production_distractor2'), axis=1)
    # same for interpretation
    df['scores_interpretation_target'] = df.apply(lambda r: avg_score(r, option='interpretation_target'), axis=1)
    df['scores_interpretation_competitor'] = df.apply(lambda r: avg_score(r, option='interpretation_competitor'), axis=1)
    df['scores_interpretation_distractor'] = df.apply(lambda r: avg_score(r, option='interpretation_distractor'), axis=1)
    
    # compute probabilities (rowwise), with alpha as stated in the file
    df = df.apply(lambda r: compute_row_probabilities(r), axis=1)

    # filter out the columns which are not needed
    cols_to_keep = list(raw_df.columns)
    df_out = df[df.columns[df.columns.isin(cols_to_keep)]]
    # save
    out_file_name = filename.split("/")[-1].split(".")[0] 
    df_out.to_csv(f"../02-data/mean_scores/{out_file_name}.csv", index=False)


In [99]:
import os
files_to_convert = ["../02-data/" + p for p in os.listdir("../02-data/") if "Llama" in p]
tokenizer_70b = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-hf")
tokenizer_70b_chat = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")
for p in files_to_convert:
    get_average_scores(p, tokenizer_70b_chat if "chat" in p else tokenizer_70b)

Len context  146
tokens shape  7
Len context  153
tokens shape  9
Len context  150
tokens shape  7
Len context  144
tokens shape  7
Len context  147
tokens shape  6
Len context  150
tokens shape  8
Len context  150
tokens shape  8
Len context  146
tokens shape  7
Len context  148
tokens shape  6
Len context  148
tokens shape  8
Len context  148
tokens shape  7
Len context  148
tokens shape  7
Len context  147
tokens shape  7
Len context  149
tokens shape  6
Len context  148
tokens shape  7
Len context  150
tokens shape  8
Len context  146
tokens shape  7
Len context  150
tokens shape  7
Len context  148
tokens shape  7
Len context  151
tokens shape  7
Len context  145
tokens shape  6
Len context  150
tokens shape  7
Len context  149
tokens shape  7
Len context  148
tokens shape  7
Len context  150
tokens shape  8
Len context  147
tokens shape  7
Len context  150
tokens shape  7
Len context  152
tokens shape  8
Len context  149
tokens shape  7
Len context  148
tokens shape  7
Len contex

In [100]:
# do the same for GPT
import tiktoken
gpt_encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
gpt_results = pd.read_csv("../02-data/results_GPT.csv")

def pull_gpt_tokens(row, encoding, option="target", task="production"):
    """
    Helper for retrieving number the number of tokens corresponding to different choice options in interpretation task.
    """
    context = row['context_interpretation'] if task == "interpretation" else row["context_production"]
    len_context = len(encoding.encode(context.strip()))
    print("Len context ", len_context)
    if task == "interpretation":
        tokens_shape = (
            len(encoding.encode(context + '"' + row["interpretation_" + option] + '"')) - len_context 
        )
    else:
        tokens_shape = (
            len(encoding.encode(context + '"' + row["production_" + option] + '"')) - len_context 
        ) 
    print("tokens shape ", tokens_shape)
    return tokens_shape

In [104]:
gpt_df = gpt_results.copy()
gpt_df.head()

,trial,objects,utterances,trigger_feature,nuisance_feature,production_target,production_competitor,production_distractor1,production_distractor2,production_index_target,...,prob_production_competitor,prob_production_distractor1,prob_production_distractor2,alpha_interpretation,scores_interpretation_target,scores_interpretation_competitor,scores_interpretation_distractor,prob_interpretation_target,prob_interpretation_competitor,prob_interpretation_distractor
0,0,a green triangle with dots\na green circle wit...,green\ntriangle\nred\ncircle,color,texture,red,triangle,green,circle,2,...,0.000797,0.000506,0.000315,0.5,-15.912748,-26.771464,-21.729583,0.944334,0.004142,0.051525
1,1,a orange triangle with stripes\na orange hexag...,triangle\nstripes\nstars\nhexagon,texture,color,stars,hexagon,stripes,triangle,2,...,0.001185,0.000304,0.000181,0.5,-30.333501,-14.033738,-36.528472,0.000289,0.999698,0.000013
2,2,a orange circle with stripes\na blue circle wi...,orange\nblue\nstripes\nspades,color,shape,orange,stripes,blue,spades,0,...,0.001483,0.000463,0.000453,0.5,-15.935874,-30.306173,-20.756016,0.916954,0.000695,0.082351
3,3,a orange square with stars\na orange triangle ...,blue\ntriangle\norange\nsquare,shape,texture,square,orange,triangle,blue,3,...,0.679919,0.002431,0.001572,0.5,-14.266216,-19.361512,-32.543217,0.927323,0.072577,0.000100
4,4,a red square with dots\na red hexagon with sta...,dots\nsquare\nhexagon\nstars,shape,color,hexagon,stars,square,dots,2,...,0.019010,0.001391,0.000164,0.5,-21.127445,-13.998826,-36.819873,0.027536,0.972453,0.000011


In [105]:
# pull token counts for each option
gpt_df["production_target_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "target", "production"), axis=1)
gpt_df["production_competitor_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "competitor", "production"), axis=1)
gpt_df["production_distractor1_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "distractor1", "production"), axis=1)
gpt_df["production_distractor2_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "distractor2", "production"), axis=1)
gpt_df["interpretation_target_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "target", "interpretation"), axis=1)
gpt_df["interpretation_competitor_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "competitor", "interpretation"), axis=1)
gpt_df["interpretation_distractor_tokens"] = gpt_df.apply(lambda r: pull_gpt_tokens(r, gpt_encoding, "distractor", "interpretation"), axis=1)

# use the numbers of tokens in respective option to compute the average scores
gpt_df['scores_production_target'] = gpt_df.apply(lambda r: avg_score(r, option='production_target'), axis=1)
gpt_df['scores_production_competitor'] = gpt_df.apply(lambda r: avg_score(r, option='production_competitor'), axis=1)
gpt_df['scores_production_distractor1'] = gpt_df.apply(lambda r: avg_score(r, option='production_distractor1'), axis=1)
gpt_df['scores_production_distractor2'] = gpt_df.apply(lambda r: avg_score(r, option='production_distractor2'), axis=1)
# same for interpretation
gpt_df['scores_interpretation_target'] = gpt_df.apply(lambda r: avg_score(r, option='interpretation_target'), axis=1)
gpt_df['scores_interpretation_competitor'] = gpt_df.apply(lambda r: avg_score(r, option='interpretation_competitor'), axis=1)
gpt_df['scores_interpretation_distractor'] = gpt_df.apply(lambda r: avg_score(r, option='interpretation_distractor'), axis=1)

# compute probabilities (rowwise), with alpha as stated in the file
gpt_df = gpt_df.apply(lambda r: compute_row_probabilities(r), axis=1)

# filter out the columns which are not needed
cols_to_keep = list(gpt_results.columns)
gpt_out = gpt_df[gpt_df.columns[gpt_df.columns.isin(cols_to_keep)]]
# save 
# gpt_out.to_csv(f"../02-data/refgame/with_quotes/mean_scores_forward/results_GPT.csv", index=False)

Len context  107
tokens shape  3
Len context  112
tokens shape  3
Len context  110
tokens shape  3
Len context  107
tokens shape  3
Len context  110
tokens shape  4
Len context  113
tokens shape  3
Len context  108
tokens shape  3
Len context  107
tokens shape  3
Len context  108
tokens shape  4
Len context  107
tokens shape  3
Len context  108
tokens shape  3
Len context  108
tokens shape  3
Len context  109
tokens shape  3
Len context  108
tokens shape  4
Len context  108
tokens shape  4
Len context  111
tokens shape  3
Len context  107
tokens shape  3
Len context  108
tokens shape  3
Len context  108
tokens shape  3
Len context  112
tokens shape  4
Len context  109
tokens shape  3
Len context  111
tokens shape  4
Len context  112
tokens shape  3
Len context  108
tokens shape  4
Len context  109
tokens shape  3
Len context  107
tokens shape  3
Len context  110
tokens shape  3
Len context  116
tokens shape  3
Len context  110
tokens shape  3
Len context  109
tokens shape  3
Len contex